# SoniTranslate

| Description | Link |
| ----------- | ---- |
| 🎉 Repository | [![GitHub Repository](https://img.shields.io/badge/GitHub-Repository-black?style=flat-square&logo=github)](https://github.com/R3gm/SoniTranslate/) |
| 🚀 Online Demo in HF | [![Hugging Face Spaces](https://img.shields.io/badge/%F0%9F%A4%97%20Hugging%20Face-Spaces-blue)](https://huggingface.co/spaces/r3gm/SoniTranslate_translate_audio_of_a_video_content) |




In [1]:
#@title [STEP 01/08] Repo owners & refs — CONFIG ONLY
#@markdown Конфигурация источников. Можно переопределить через ENV переменные.

OWNER = {"original": "R3gm", "fork": "tekswirl25"}
USE = "fork"  #@param ["original", "fork"]

REPOS = {
    "sonitranslate": {"name": "SoniTranslate",   "ref": "main"},
    "whisperx":      {"name": "whisperX",        "ref": "cuda_12_x"},
    "pyannote":      {"name": "pyannote-audio",  "ref": "3.1.1"},
}

import os, json

for key, info in REPOS.items():
    PREFIX = key.upper()
    if not os.environ.get(f"{PREFIX}_URL"):
        os.environ[f"{PREFIX}_URL"] = f"https://github.com/{OWNER[USE]}/{info['name']}.git"
    if not os.environ.get(f"{PREFIX}_REF"):
        os.environ[f"{PREFIX}_REF"] = info["ref"]
    # совместимость: BRANCH алиас к REF
    os.environ[f"{PREFIX}_BRANCH"] = os.environ[f"{PREFIX}_REF"]

print("MODE:", USE, "| OWNER:", OWNER[USE])
print(json.dumps({
    "SONITRANSLATE": [os.environ["SONITRANSLATE_URL"], os.environ["SONITRANSLATE_REF"]],
    "WHISPERX":      [os.environ["WHISPERX_URL"],      os.environ["WHISPERX_REF"]],
    "PYANNOTE":      [os.environ["PYANNOTE_URL"],      os.environ["PYANNOTE_REF"]],
}, indent=2, ensure_ascii=False))


MODE: fork | OWNER: tekswirl25
{
  "SONITRANSLATE": [
    "https://github.com/tekswirl25/SoniTranslate.git",
    "main"
  ],
  "WHISPERX": [
    "https://github.com/tekswirl25/whisperX.git",
    "cuda_12_x"
  ],
  "PYANNOTE": [
    "https://github.com/tekswirl25/pyannote-audio.git",
    "3.1.1"
  ]
}


In [2]:
%%bash
# [STEP 02/08] Quick echo — human check (idempotent)
set -euo pipefail
LOG_DIR="${LOG_DIR:-/content/_install_logs}"
mkdir -p "$LOG_DIR"

{
  echo "SONITRANSLATE: ${SONITRANSLATE_URL:-unset} @ ${SONITRANSLATE_REF:-unset}"
  echo "WHISPERX:      ${WHISPERX_URL:-unset}      @ ${WHISPERX_REF:-unset}"
  echo "PYANNOTE:      ${PYANNOTE_URL:-unset}      @ ${PYANNOTE_REF:-unset}"
} | tee "$LOG_DIR/02_echo.txt"


SONITRANSLATE: https://github.com/tekswirl25/SoniTranslate.git @ main
WHISPERX:      https://github.com/tekswirl25/whisperX.git      @ cuda_12_x
PYANNOTE:      https://github.com/tekswirl25/pyannote-audio.git      @ 3.1.1


In [3]:
%%bash
# [STEP 03/08] Remote ref validation — branch/tag auto-detect, ENV-compatible
set -euo pipefail
LOG_DIR="${LOG_DIR:-/content/_install_logs}"
mkdir -p "$LOG_DIR"

check_ref () {
  local url="$1" ref="$2" type="${3:-}"
  local name
  name="$(basename "$url" .git)"

  # Helper checks
  local tag_ok=1 head_ok=1

  if [[ -n "$type" ]]; then
    # Respect explicit type if provided
    if [[ "$type" == "tag" ]]; then
      if git ls-remote --tags "$url" "refs/tags/${ref}" >/dev/null 2>&1; then
        echo "[OK] ${name}: tag '${ref}' found"
      else
        echo "[FAIL] ${name}: tag '${ref}' NOT found"
      fi
    else
      if git ls-remote --heads "$url" "$ref" >/dev/null 2>&1; then
        echo "[OK] ${name}: branch '${ref}' found"
      else
        echo "[FAIL] ${name}: branch '${ref}' NOT found"
      fi
    fi
    return 0
  fi

  # Auto-detect when REFTYPE is not provided
  git ls-remote --tags  "$url" "refs/tags/${ref}" >/dev/null 2>&1 && tag_ok=0 || tag_ok=1
  git ls-remote --heads "$url" "$ref"              >/dev/null 2>&1 && head_ok=0 || head_ok=1

  if [[ $tag_ok -eq 0 && $head_ok -eq 0 ]]; then
    echo "[OK] ${name}: ref '${ref}' exists as BOTH (tag & branch)"
  elif [[ $tag_ok -eq 0 ]]; then
    echo "[OK] ${name}: tag '${ref}' found"
  elif [[ $head_ok -eq 0 ]]; then
    echo "[OK] ${name}: branch '${ref}' found"
  else
    echo "[FAIL] ${name}: ref '${ref}' not found as tag or branch"
  fi
}

{
  check_ref "${SONITRANSLATE_URL:-}" "${SONITRANSLATE_REF:-}" "${SONITRANSLATE_REFTYPE:-}"
  check_ref "${WHISPERX_URL:-}"      "${WHISPERX_REF:-}"      "${WHISPERX_REFTYPE:-}"
  check_ref "${PYANNOTE_URL:-}"      "${PYANNOTE_REF:-}"      "${PYANNOTE_REFTYPE:-}"
} | tee "$LOG_DIR/03_validate_refs.txt"

[OK] SoniTranslate: ref 'main' exists as BOTH (tag & branch)
[OK] whisperX: ref 'cuda_12_x' exists as BOTH (tag & branch)
[OK] pyannote-audio: ref '3.1.1' exists as BOTH (tag & branch)


In [4]:
%%bash
# [STEP 04/08] SAFE CLONE & REQUIREMENTS PREVIEW (NO INSTALL)
set -euo pipefail
LOG_DIR="${LOG_DIR:-/content/_install_logs}"
mkdir -p "$LOG_DIR"

REPO_DIR="/content/SoniTranslate_debug"
URL="${SONITRANSLATE_URL:-}"
REF="${SONITRANSLATE_REF:-}"
REQ="requirements_base.txt"

{
  echo "== repo: $URL @ $REF =="

  # fresh shallow clone
  rm -rf "$REPO_DIR"
  git clone --depth=2 "$URL" "$REPO_DIR" -q
  cd "$REPO_DIR"

  # fetch + checkout supports both tag and branch safely
  git fetch --depth=2 origin "$REF" -q || true
  if git rev-parse --verify -q "refs/remotes/origin/$REF" >/dev/null; then
    git checkout -qf "origin/$REF"
  elif git rev-parse --verify -q "refs/tags/$REF" >/dev/null; then
    git checkout -qf "refs/tags/$REF"
  else
    echo "[WARN] ref '$REF' not found as branch or tag; staying on default clone HEAD"
  fi

  if [[ ! -f "$REQ" ]]; then
    echo "[info] '$REQ' not found — nothing to preview."
    exit 0
  fi

  echo "== current whisperX lines in ${REQ} =="
  grep -n -E 'git\+https://github\.com/.*/whisperX\.git@.*' "$REQ" || echo "[info] whisperX line not found"

  echo "== current pyannote lines in ${REQ} =="
  grep -n -E 'git\+https://github\.com/.*/pyannote-audio\.git@.*' "$REQ" || echo "[info] pyannote line not found"

  cp "$REQ" "${REQ}.preview"

  # Substitute to ENV-refs in preview only (no deps install here)
  sed -i "s|git+https://github.com/.*/whisperX.git@.*|git+${WHISPERX_URL:-https://github.com/placeholder/whisperX.git}@${WHISPERX_REF:-main}|" "${REQ}.preview"
  sed -i "s|git+https://github.com/.*/pyannote-audio.git@.*|git+${PYANNOTE_URL:-https://github.com/placeholder/pyannote-audio.git}@${PYANNOTE_REF:-3.1.1}|" "${REQ}.preview"

  echo "== PREVIEW DIFF (original vs preview) =="
  diff -u "$REQ" "${REQ}.preview" || true

  rm -f "${REQ}.preview"
  echo "[done] preview only; original requirements not modified."
} | tee "$LOG_DIR/04_clone_preview.txt"



== repo: https://github.com/tekswirl25/SoniTranslate.git @ main ==
== current whisperX lines in requirements_base.txt ==
12:git+https://github.com/R3gm/whisperX.git@cuda_11_8
== current pyannote lines in requirements_base.txt ==
11:git+https://github.com/R3gm/pyannote-audio.git@3.1.1
== PREVIEW DIFF (original vs preview) ==
--- requirements_base.txt	2025-08-27 19:34:33.752694937 +0000
+++ requirements_base.txt.preview	2025-08-27 19:34:34.398752992 +0000
@@ -8,8 +8,8 @@
 pydub==0.25.1
 edge_tts==6.1.7
 deep_translator==1.11.4
-git+https://github.com/R3gm/pyannote-audio.git@3.1.1
-git+https://github.com/R3gm/whisperX.git@cuda_11_8
+git+https://github.com/tekswirl25/pyannote-audio.git@3.1.1
+git+https://github.com/tekswirl25/whisperX.git@cuda_12_x
 nest_asyncio
 gTTS
 gradio_client==0.10.1
[done] preview only; original requirements not modified.


In [5]:
%%bash
# [STEP 05/08] SCAN REQUIREMENTS (READ-ONLY AUDIT)
set -euo pipefail

rm -rf SoniTranslate_scan
git clone -q --depth=2 "${SONITRANSLATE_URL}" SoniTranslate_scan
cd SoniTranslate_scan
git fetch -q --depth=2 origin "${SONITRANSLATE_REF}" || true
git checkout -qf "origin/${SONITRANSLATE_REF}" 2>/dev/null || git checkout -qf "refs/tags/${SONITRANSLATE_REF}" 2>/dev/null || echo "[WARN] ref not found; using default HEAD"

echo "== FILES =="
find . -maxdepth 2 -type f -name "requirements*.txt" -printf "%P\n" | sort || true

echo -e "\n== GREP: torch with +cu suffix =="
grep -nE '^torch[^#]*\+cu[0-9_]+' requirements*.txt */requirements*.txt 2>/dev/null || echo "[ok] no '+cu' torch pins found"

echo -e "\n== GREP: TTS==0.21.1 =="
grep -nE '(^|[^A-Za-z])TTS==0\.21\.1([^A-Za-z]|$)' requirements*.txt */requirements*.txt 2>/dev/null || echo "[ok] no TTS==0.21.1 pins"

echo -e "\n== GREP: whisperX git lines =="
grep -nE 'git\+https://github\.com/.*/whisperX\.git@.*' requirements*.txt */requirements*.txt 2>/dev/null || echo "[info] no whisperX git lines found"

echo -e "\n== GREP: websockets/opencv hard pins (for awareness) =="
grep -nE 'websockets|opencv-python' requirements*.txt */requirements*.txt 2>/dev/null || echo "[info] none"



== FILES ==
requirements_base.txt
requirements_extra.txt
requirements.txt
requirements_xtts.txt

== GREP: torch with +cu suffix ==
requirements_base.txt:2:torch>=2.1.0+cu118
requirements_base.txt:3:torchvision>=0.16.0+cu118
requirements_base.txt:4:torchaudio>=2.1.0+cu118
requirements.txt:4:torch>=2.1.0+cu118
requirements.txt:5:torchvision>=0.16.0+cu118
requirements.txt:6:torchaudio>=2.1.0+cu118
[ok] no '+cu' torch pins found

== GREP: TTS==0.21.1 ==
requirements_xtts.txt:58:# pip install TTS==0.21.1  --no-deps
[ok] no TTS==0.21.1 pins

== GREP: whisperX git lines ==
requirements_base.txt:12:git+https://github.com/R3gm/whisperX.git@cuda_11_8
requirements.txt:12:git+https://github.com/m-bain/whisperX.git@a5dca2c
[info] no whisperX git lines found

== GREP: websockets/opencv hard pins (for awareness) ==
[info] none


In [6]:
%%bash
# [STEP 06/08] PREVIEW FIXES (NO WRITE)
set -euo pipefail

rm -rf SoniTranslate_fixpreview
git clone -q --depth=2 "${SONITRANSLATE_URL}" SoniTranslate_fixpreview
cd SoniTranslate_fixpreview
git fetch --depth=2 origin "${SONITRANSLATE_REF}" -q || true
git checkout -qf "origin/${SONITRANSLATE_REF}" 2>/dev/null || git checkout -qf "refs/tags/${SONITRANSLATE_REF}" 2>/dev/null || echo "[WARN] ref not found; using default HEAD"

fix_one() {
  local file="$1"
  [[ -f "$file" ]] || return 0
  cp "$file" "${file}.preview"

  # 1) torch с суффиксом +cu... → заменить на просто 'torch'
  sed -i -E 's/^torch[^#]*\+cu[0-9_]+/torch/' "${file}.preview"

  # 2) TTS==0.21.1 → диапазон для colab/python3.12
  sed -i -E 's/(^|[^A-Za-z])TTS==0\.21\.1([^A-Za-z]|$)/TTS>=0.22,<0.23/g' "${file}.preview"

  echo "### DIFF for $file"
  diff -u "$file" "${file}.preview" || true
  rm -f "${file}.preview"
}

for f in requirements*.txt; do fix_one "$f"; done
echo "[done] only preview; no files modified."



### DIFF for requirements_base.txt
--- requirements_base.txt	2025-08-27 19:34:36.884976405 +0000
+++ requirements_base.txt.preview	2025-08-27 19:34:37.355018643 +0000
@@ -1,7 +1,7 @@
 --extra-index-url https://download.pytorch.org/whl/cu118
-torch>=2.1.0+cu118
-torchvision>=0.16.0+cu118
-torchaudio>=2.1.0+cu118
+torch
+torch
+torch
 ctranslate2<=4.4.0
 yt-dlp
 gradio==4.19.2
### DIFF for requirements_extra.txt
### DIFF for requirements.txt
--- requirements.txt	2025-08-27 19:34:36.884976405 +0000
+++ requirements.txt.preview	2025-08-27 19:34:37.372020171 +0000
@@ -1,9 +1,9 @@
 # Temporal requirements
 nest_asyncio
 --extra-index-url https://download.pytorch.org/whl/cu118
-torch>=2.1.0+cu118
-torchvision>=0.16.0+cu118
-torchaudio>=2.1.0+cu118
+torch
+torch
+torch
 yt-dlp
 gradio==4.19.2
 pydub==0.25.1
### DIFF for requirements_xtts.txt
--- requirements_xtts.txt	2025-08-27 19:34:36.885976495 +0000
+++ requirements_xtts.txt.preview	2025-08-27 19:34:37.380020889 +0000
@@ -55,4 +55,4 @@
 
 #

In [7]:
%%bash
# [STEP 07/08] CREATE CONSTRAINTS.TXT (READABLE PINS)
cat > /content/constraints_sonitranslate.txt <<'TXT'
# soft constraints to avoid common conflicts (used with --constraint)
# torch ставим отдельно до requirements, поэтому здесь его нет

scipy>=1.11
websockets>=15,<16
opencv-python==4.10.0.84
TTS>=0.22,<0.23
transformers>=4.41
sentence-transformers>=3.0
gradio
TXT

echo "== constraints_sonitranslate.txt =="
cat /content/constraints_sonitranslate.txt


== constraints_sonitranslate.txt ==
# soft constraints to avoid common conflicts (used with --constraint)
# torch ставим отдельно до requirements, поэтому здесь его нет

scipy>=1.11
websockets>=15,<16
opencv-python==4.10.0.84
TTS>=0.22,<0.23
transformers>=4.41
sentence-transformers>=3.0
gradio


In [8]:
#@title [STEP 07MODE/08] NUMPY ↔ GRADIO PROFILE (CONFIG ONLY)
#@markdown Профили:
#@markdown • **upstream** — как в оригинале (gradio 4.19.2 → numpy 1.26.4)
#@markdown • **modern** — эксперимент c NumPy 2.x (gradio ≥ 4.30)
PROFILE = "modern"  #@param ["upstream", "modern"]

if PROFILE == "upstream":
    NUMPY_SPEC  = "numpy==1.26.4"
    GRADIO_SPEC = "gradio==4.19.2"
elif PROFILE == "modern":
    NUMPY_SPEC  = "numpy>=2.1,<2.3"
    GRADIO_SPEC = "gradio>=4.30"
else:
    raise ValueError("PROFILE must be 'upstream' or 'modern'")

import os, json
# уважаем внешние overrides, если они уже выставлены
os.environ.setdefault("NUMPY_SPEC",  NUMPY_SPEC)
os.environ.setdefault("GRADIO_SPEC", GRADIO_SPEC)
os.environ.setdefault("PROFILE",     PROFILE)

print("PROFILE:", os.environ["PROFILE"])
print(json.dumps({
    "NUMPY_SPEC":  os.environ["NUMPY_SPEC"],
    "GRADIO_SPEC": os.environ["GRADIO_SPEC"]
}, indent=2))



PROFILE: modern
{
  "NUMPY_SPEC": "numpy>=2.1,<2.3",
  "GRADIO_SPEC": "gradio>=4.30"
}


In [9]:
# [STEP 07AUTO/08] PY VERSION AWARE NUMPY/GRADIO (CONFIG ONLY, NON-OVERRIDING)
import sys, os, json, re

py = sys.version_info
py_str = f"{py.major}.{py.minor}.{py.micro}"
print("Detected Python:", py_str)

# Берём то, что уже задал профиль (07MODE). Ничего не перезаписываем.
NUMPY_SPEC  = os.environ.get("NUMPY_SPEC")   # может быть None
GRADIO_SPEC = os.environ.get("GRADIO_SPEC")  # может быть None

# Если профиль ничего не задал — подберём дефолты под версию Python.
# Примечание: NumPy 2.x поддерживает Python ≥3.9, поэтому 3.12 ОК и для 1.26.4, и для 2.x.
if not NUMPY_SPEC:
    if (py.major, py.minor) >= (3, 11):
        # современный дефолт; безопасен для 3.11+ и не конфликтует с большинством стеков
        NUMPY_SPEC = "numpy>=2.1,<2.3"
    else:
        NUMPY_SPEC = "numpy==1.26.4"

if not GRADIO_SPEC:
    # дефолт «как в апстриме», если профиль его не задал
    GRADIO_SPEC = "gradio==4.19.2"

# Лёгкая диагностика потенциальных несостыковок (только предупреждаем)
def major_ver(spec: str, name: str):
    if not spec: return None
    m = re.search(rf"{name}\s*([<>=!~]=\s*)?(\d+)\.(\d+)", spec.replace(" ", ""), re.I)
    return int(m.group(2)) if m else None

np_major = major_ver(NUMPY_SPEC, "numpy")
gr_set   = bool(GRADIO_SPEC)

if np_major == 2 and GRADIO_SPEC == "gradio==4.19.2":
    print("[WARN] NumPy 2.x с gradio==4.19.2: если поймаете конфликт, поднимите gradio (например, '>=4.30') или верните NumPy 1.26.4.")

# Экспорт без перезаписи ранее заданных — используем уже рассчитанные значения
os.environ["NUMPY_SPEC"]  = NUMPY_SPEC
os.environ["GRADIO_SPEC"] = GRADIO_SPEC

print(json.dumps({"Python": py_str, "NUMPY_SPEC": NUMPY_SPEC, "GRADIO_SPEC": GRADIO_SPEC}, indent=2))



Detected Python: 3.12.11
{
  "Python": "3.12.11",
  "NUMPY_SPEC": "numpy>=2.1,<2.3",
  "GRADIO_SPEC": "gradio>=4.30"
}


In [10]:
# [STEP 07DBG/08] DEBUG PROFILE — manual toggles for secondary deps
import os, json

# === двойные строки: можно раскомментировать хардкод и закомментировать ENV ===

# SCIPY_SPEC        = os.environ.get("SCIPY_SPEC")
SCIPY_SPEC        = "scipy>=1.11"  # для NumPy 2.x (Py3.12 можно и >=1.13)

# WEBSOCKETS_SPEC   = os.environ.get("WEBSOCKETS_SPEC")
# WEBSOCKETS_SPEC   = "websockets>=10,<12"
WEBSOCKETS_SPEC   = "-"   # спец-значение «не писать в constraints»

# OPENCV_SPEC       = os.environ.get("OPENCV_SPEC")
OPENCV_SPEC       = "opencv-python==4.10.0.84"

# TTS_SPEC          = os.environ.get("TTS_SPEC")
TTS_SPEC          = "TTS>=0.22,<0.23"

# TRANSFORMERS_SPEC = os.environ.get("TRANSFORMERS_SPEC")
# TRANSFORMERS_SPEC = "transformers>=4.41"
TRANSFORMERS_SPEC = "transformers>=4.33,<4.37"  # дружит с tokenizers<0.16

# SENT_TR_SPEC      = os.environ.get("SENTENCE_TRANSFORMERS_SPEC")
SENT_TR_SPEC      = "sentence-transformers>=2.2,<3.0"  # v3 требует transformers>=4.41

# TOKENIZERS_SPEC   = os.environ.get("TOKENIZERS_SPEC")
TOKENIZERS_SPEC   = "tokenizers>=0.13,<0.16"    # под faster-whisper==1.0.0

# === TORCH trio (torch/torchvision/torchaudio) ===
# TORCH_SPEC       = os.environ.get("TORCH_SPEC")
TORCH_SPEC       = "torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1"

# TORCH_INDEX_URL  = os.environ.get("TORCH_INDEX_URL")
TORCH_INDEX_URL  = "https://download.pytorch.org/whl/cpu"
# TORCH_INDEX_URL  = "https://download.pytorch.org/whl/cu121"   # пример для CUDA 12.1

# OMEGACONF/HYDRA/FAIRSEQ для requirements_extra
# OMEGACONF_SPEC   = os.environ.get("OMEGACONF_SPEC")
OMEGACONF_SPEC   = "omegaconf==2.0.6"
# HYDRA_CORE_SPEC  = os.environ.get("HYDRA_CORE_SPEC")
HYDRA_CORE_SPEC  = "hydra-core==1.0.7"
# FAIRSEQ_SPEC     = os.environ.get("FAIRSEQ_SPEC")
FAIRSEQ_SPEC     = "fairseq==0.12.2"


def set_if_nonempty(key, val):
    """Write to ENV only if val is non-empty and not '-' (our 'skip' marker)."""
    if isinstance(val, str):
        v = val.strip()
        if v and v != "-":
            os.environ[key] = v

# === записываем выбранные в ENV (только непустые строки, без '-') ===
set_if_nonempty("SCIPY_SPEC",        SCIPY_SPEC)
set_if_nonempty("WEBSOCKETS_SPEC",   WEBSOCKETS_SPEC)
set_if_nonempty("OPENCV_SPEC",       OPENCV_SPEC)
set_if_nonempty("TTS_SPEC",          TTS_SPEC)
set_if_nonempty("TRANSFORMERS_SPEC", TRANSFORMERS_SPEC)
set_if_nonempty("SENTENCE_TRANSFORMERS_SPEC", SENT_TR_SPEC)
set_if_nonempty("TOKENIZERS_SPEC",   TOKENIZERS_SPEC)
set_if_nonempty("TORCH_SPEC",        TORCH_SPEC)
set_if_nonempty("TORCH_INDEX_URL",   TORCH_INDEX_URL)
set_if_nonempty("OMEGACONF_SPEC",    OMEGACONF_SPEC)
set_if_nonempty("HYDRA_CORE_SPEC",   HYDRA_CORE_SPEC)
set_if_nonempty("FAIRSEQ_SPEC",      FAIRSEQ_SPEC)

print("== DEBUG PROFILE (effective) ==")
print(json.dumps({
    "NUMPY":  os.environ.get("NUMPY_SPEC"),    # управляется через 07MODE/07AUTO
    "GRADIO": os.environ.get("GRADIO_SPEC"),   # управляется через 07MODE/07AUTO
    "SCIPY":  os.environ.get("SCIPY_SPEC"),
    "WEBSOCKETS": os.environ.get("WEBSOCKETS_SPEC"),  # будет None, если "-"
    "OPENCV": os.environ.get("OPENCV_SPEC"),
    "TTS": os.environ.get("TTS_SPEC"),
    "TRANSFORMERS": os.environ.get("TRANSFORMERS_SPEC"),
    "SENTENCE-TRANSFORMERS": os.environ.get("SENTENCE_TRANSFORMERS_SPEC"),
    "TOKENIZERS_SPEC": os.environ.get("TOKENIZERS_SPEC"),
    "TORCH_SPEC": os.environ.get("TORCH_SPEC"),
    "TORCH_INDEX_URL": os.environ.get("TORCH_INDEX_URL"),
    "OMEGACONF_SPEC": os.environ.get("OMEGACONF_SPEC"),
    "HYDRA_CORE_SPEC": os.environ.get("HYDRA_CORE_SPEC"),
    "FAIRSEQ_SPEC": os.environ.get("FAIRSEQ_SPEC"),
}, indent=2))


== DEBUG PROFILE (effective) ==
{
  "NUMPY": "numpy>=2.1,<2.3",
  "GRADIO": "gradio>=4.30",
  "SCIPY": "scipy>=1.11",
  "WEBSOCKETS": null,
  "OPENCV": "opencv-python==4.10.0.84",
  "TTS": "TTS>=0.22,<0.23",
  "TRANSFORMERS": "transformers>=4.33,<4.37",
  "SENTENCE-TRANSFORMERS": "sentence-transformers>=2.2,<3.0",
  "TOKENIZERS_SPEC": "tokenizers>=0.13,<0.16",
  "TORCH_SPEC": "torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1",
  "TORCH_INDEX_URL": "https://download.pytorch.org/whl/cpu",
  "OMEGACONF_SPEC": "omegaconf==2.0.6",
  "HYDRA_CORE_SPEC": "hydra-core==1.0.7",
  "FAIRSEQ_SPEC": "fairseq==0.12.2"
}


In [11]:
%%bash
# [STEP 07CONS/08] WRITE CONSTRAINTS FROM ENV (STRICT FOR NUMPY/GRADIO)
set -euo pipefail

: "${NUMPY_SPEC?NUMPY_SPEC is not set (run 07MODE/07AUTO).}"
: "${GRADIO_SPEC?GRADIO_SPEC is not set (run 07MODE/07AUTO).}"

# читаем опциональные (безопасно при set -u)
: "${SCIPY_SPEC:=}"
: "${WEBSOCKETS_SPEC:=}"
: "${OPENCV_SPEC:=}"
: "${TTS_SPEC:=}"
: "${TRANSFORMERS_SPEC:=}"
: "${SENTENCE_TRANSFORMERS_SPEC:=}"
: "${TOKENIZERS_SPEC:=}"
: "${OMEGACONF_SPEC:=}"
: "${HYDRA_CORE_SPEC:=}"
: "${FAIRSEQ_SPEC:=}"
: "${GRADIO_CLIENT_SPEC:=}"   # опционально

CONS="/content/constraints_sonitranslate.txt"
{
  echo "${NUMPY_SPEC}"
  [[ -n "${SCIPY_SPEC}"                  && "${SCIPY_SPEC}"                  != "-" ]] && echo "${SCIPY_SPEC}"
  [[ -n "${WEBSOCKETS_SPEC}"             && "${WEBSOCKETS_SPEC}"             != "-" ]] && echo "${WEBSOCKETS_SPEC}"
  [[ -n "${OPENCV_SPEC}"                 && "${OPENCV_SPEC}"                 != "-" ]] && echo "${OPENCV_SPEC}"
  [[ -n "${TTS_SPEC}"                    && "${TTS_SPEC}"                    != "-" ]] && echo "${TTS_SPEC}"
  [[ -n "${TRANSFORMERS_SPEC}"           && "${TRANSFORMERS_SPEC}"           != "-" ]] && echo "${TRANSFORMERS_SPEC}"
  [[ -n "${SENTENCE_TRANSFORMERS_SPEC}"  && "${SENTENCE_TRANSFORMERS_SPEC}"  != "-" ]] && echo "${SENTENCE_TRANSFORMERS_SPEC}"
  [[ -n "${TOKENIZERS_SPEC}"             && "${TOKENIZERS_SPEC}"             != "-" ]] && echo "${TOKENIZERS_SPEC}"
  [[ -n "${OMEGACONF_SPEC}"              && "${OMEGACONF_SPEC}"              != "-" ]] && echo "${OMEGACONF_SPEC}"
  [[ -n "${HYDRA_CORE_SPEC}"             && "${HYDRA_CORE_SPEC}"             != "-" ]] && echo "${HYDRA_CORE_SPEC}"
  [[ -n "${FAIRSEQ_SPEC}"                && "${FAIRSEQ_SPEC}"                != "-" ]] && echo "${FAIRSEQ_SPEC}"
  [[ -n "${GRADIO_CLIENT_SPEC}"          && "${GRADIO_CLIENT_SPEC}"          != "-" ]] && echo "${GRADIO_CLIENT_SPEC}"
  echo "${GRADIO_SPEC}"
} > "$CONS"

echo "== USING CONSTRAINTS (PROFILE=${PROFILE:-unknown}) =="
cat "$CONS"


== USING CONSTRAINTS (PROFILE=modern) ==
numpy>=2.1,<2.3
scipy>=1.11
opencv-python==4.10.0.84
TTS>=0.22,<0.23
transformers>=4.33,<4.37
sentence-transformers>=2.2,<3.0
tokenizers>=0.13,<0.16
omegaconf==2.0.6
hydra-core==1.0.7
fairseq==0.12.2
gradio>=4.30


In [12]:
%%bash
# [STEP 08A/08] PRECHECK UV & GIT (NO INSTALL)
set -euo pipefail

echo "== python/pip/uv/git versions =="
python -V
python -m pip -V
git --version || true
python -c "import shutil; print('uv on PATH:', bool(shutil.which('uv')))"

echo "== installing uv if missing =="
python -m pip install -q --upgrade pip setuptools wheel
python -m pip install -q uv
python -c "import shutil; print('uv on PATH (after):', bool(shutil.which('uv')))"

echo "== env =="
echo "SONITRANSLATE_URL = ${SONITRANSLATE_URL}"
echo "SONITRANSLATE_REF = ${SONITRANSLATE_REF}"
echo "WHISPERX_URL      = ${WHISPERX_URL}"
echo "WHISPERX_REF      = ${WHISPERX_REF}"

echo "== remote ref check =="
git ls-remote --tags  "${SONITRANSLATE_URL}" "${SONITRANSLATE_REF}" || git ls-remote --heads "${SONITRANSLATE_URL}" "${SONITRANSLATE_REF}" || echo "[warn] ref not found"
git ls-remote --tags  "${WHISPERX_URL}"      "${WHISPERX_REF}"      || git ls-remote --heads "${WHISPERX_URL}"      "${WHISPERX_REF}"      || echo "[warn] ref not found"

echo "[ok] precheck done"


== python/pip/uv/git versions ==
Python 3.12.11
pip 24.1.2 from /usr/local/lib/python3.12/dist-packages/pip (python 3.12)
git version 2.34.1
uv on PATH: True
== installing uv if missing ==
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 48.5 MB/s eta 0:00:00
uv on PATH (after): True
== env ==
SONITRANSLATE_URL = https://github.com/tekswirl25/SoniTranslate.git
SONITRANSLATE_REF = main
WHISPERX_URL      = https://github.com/tekswirl25/whisperX.git
WHISPERX_REF      = cuda_12_x
== remote ref check ==
[ok] precheck done


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


In [13]:
%%bash
# [STEP 08AA/08] QUICK FIX: satisfy IPython 7.x -> jedi>=0.16
set -euo pipefail

# Разрешим ненулевой код у python-блока, чтобы поймать его вручную:
set +e
python - <<'PY'
import sys
try:
    import IPython
    ver = getattr(IPython, "__version__", "0")
    major = int(ver.split(".")[0])
    if major == 7:
        try:
            import jedi  # ok
            print("IPython 7.x: jedi present -> skip install")
            sys.exit(0)
        except Exception:
            print("IPython 7.x: jedi missing -> need install")
            sys.exit(42)
    else:
        print(f"IPython {ver}: not 7.x -> skip install")
        sys.exit(0)
except Exception as e:
    print(f"[warn] IPython check failed: {e}")
    sys.exit(0)
PY
rc=$?
set -e

if [[ $rc -eq 42 ]]; then
  python -m pip install -q "jedi>=0.16"
  echo "[ok] installed jedi (for IPython 7.x)"
else
  echo "[skip] no jedi install needed"
fi



IPython 7.x: jedi missing -> need install
[ok] installed jedi (for IPython 7.x)


In [14]:
%%bash
# [PREPATCH] fetch, patch & install wheel: omegaconf==2.0.6 (pip>=24.1-safe)
set -euo pipefail

PKG="omegaconf"
VER="2.0.6"
WORKDIR="/content/_patch_${PKG}_${VER}"
WHEEL_DIR="/content/_wheels"

echo "== prep =="
rm -rf "$WORKDIR" "$WHEEL_DIR"
mkdir -p "$WORKDIR" "$WHEEL_DIR"
cd "$WORKDIR"

echo "== ensure tools =="
python -m pip install -q --upgrade pip wheel

echo "== discover wheel URL from PyPI simple index =="
curl -fsSL "https://pypi.org/simple/${PKG}/" -o index.html
WHEEL_URL="$(python - <<'PY'
import re, html
p = open("index.html","r",encoding="utf-8",errors="ignore").read()
m = re.search(r'href="([^"]*omegaconf-2\.0\.6-py3-none-any\.whl[^"]*)"', p, re.I)
print(html.unescape(m.group(1)) if m else "")
PY
)"
[[ -n "$WHEEL_URL" ]] || { echo "[err] wheel URL not found"; exit 1; }
case "$WHEEL_URL" in http*) : ;; *) WHEEL_URL="https://files.pythonhosted.org/${WHEEL_URL#*/files.pythonhosted.org/}";; esac
echo "wheel url: $WHEEL_URL"

echo "== download wheel =="
curl -fSLo "${PKG}-${VER}-py3-none-any.orig.whl" "$WHEEL_URL"

echo "== unpack with wheel tool =="
python -m wheel unpack "${PKG}-${VER}-py3-none-any.orig.whl" -d "$WORKDIR/unpacked"
TARGET_DIR="$(ls -d "$WORKDIR"/unpacked/${PKG}-${VER} 2>/dev/null)"
[[ -n "${TARGET_DIR:-}" ]] || { echo "[err] unpacked dir not found"; exit 1; }

echo "== patch METADATA (PyYAML >=5.1.* -> >=5.1) =="
META_PATH="$(find "$TARGET_DIR" -maxdepth 2 -type f -path "*/${PKG}-${VER}.dist-info/METADATA" | head -n1)"
[[ -n "${META_PATH:-}" ]] || { echo "[err] METADATA not found"; exit 1; }
# до патча: покажем строки с PyYAML
echo "-- BEFORE --"; grep -E '^Requires-Dist: PyYAML' "$META_PATH" || true
# патчим
sed -i -E 's/PyYAML[[:space:]]*\(>=[[:space:]]*5\.1\.\*\)/PyYAML (>=5.1)/g' "$META_PATH"
echo "-- AFTER  --"; grep -E '^Requires-Dist: PyYAML' "$META_PATH" || true

echo "== repack with wheel tool (updates RECORD) =="
python -m wheel pack "$TARGET_DIR" -d "$WORKDIR"
WHL_PATCHED="$(ls -1 "$WORKDIR"/${PKG}-${VER}-*.whl | grep -v '\.orig\.whl$' | head -n1)"
[[ -n "${WHL_PATCHED:-}" ]] || { echo "[err] patched wheel not created"; exit 1; }
ls -l "$WHL_PATCHED"

echo "== install patched wheel =="
# ВАЖНО: ставим патченный, НЕ .orig
python -m pip install -v --no-deps "$WHL_PATCHED"

echo "== verify =="
python - <<'PY'
import omegaconf
print("omegaconf:", omegaconf.__version__)
print("ok:", omegaconf.__version__=="2.0.6")
PY

echo "== cache patched wheel to /content/_wheels =="
cp -f "$WHL_PATCHED" "$WHEEL_DIR/omegaconf-${VER}-py3-none-any.whl"
sha256sum "$WHEEL_DIR/omegaconf-${VER}-py3-none-any.whl" | tee "$WHEEL_DIR/omegaconf-${VER}-py3-none-any.whl.sha256" >/dev/null
echo "Wheel saved to: $WHEEL_DIR"
echo 'Hint for STEP 08: export PIP_FIND_LINKS="/content/_wheels${PIP_FIND_LINKS:+ $PIP_FIND_LINKS}"'


== prep ==
== ensure tools ==
== discover wheel URL from PyPI simple index ==
wheel url: https://files.pythonhosted.org/packages/d0/eb/9d63ce09dd8aa85767c65668d5414958ea29648a0eec80a4a7d311ec2684/omegaconf-2.0.6-py3-none-any.whl#sha256=9e349fd76819b95b47aa628edea1ff83fed5b25108608abdd6c7fdca188e302a
== download wheel ==
== unpack with wheel tool ==
Unpacking to: /content/_patch_omegaconf_2.0.6/unpacked/omegaconf-2.0.6...OK
== patch METADATA (PyYAML >=5.1.* -> >=5.1) ==
-- BEFORE --
Requires-Dist: PyYAML (>=5.1.*)
-- AFTER  --
Requires-Dist: PyYAML (>=5.1)
== repack with wheel tool (updates RECORD) ==
Repacking wheel as /content/_patch_omegaconf_2.0.6/omegaconf-2.0.6-py3-none-any.whl...OK
-rw-r--r-- 1 root root 36834 Aug 27 19:35 /content/_patch_omegaconf_2.0.6/omegaconf-2.0.6-py3-none-any.whl
== install patched wheel ==
Using pip 25.2 from /usr/local/lib/python3.12/dist-packages/pip (python 3.12)
Processing ./omegaconf-2.0.6-py3-none-any.whl
  Attempting uninstall: omegaconf
    Found 

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 36834  100 36834    0     0   118k      0 --:--:-- --:--:-- --:--:--  118k


In [15]:
%%bash
# [STEP 08/08] DRY INSTALL ON CPU (ISOLATED COPY, VERBOSE LOGS)
# profile-driven; pins только из 07MODE/07AUTO/07CONS; без хардкодов здесь
set -euo pipefail

LOG_DIR="/content/_install_logs"

# --- ЛОГИ: ротация (по желанию) и ЧИСТЫЙ СТАРТ ---
# ROTATE_LOGS=1 сохранит предыдущую папку логов со штампом времени
if [[ "${ROTATE_LOGS:-0}" = "1" && -d "$LOG_DIR" ]]; then
  ts="$(date +%Y%m%d_%H%M%S)"
  mv "$LOG_DIR" "${LOG_DIR}_$ts" || true
fi
rm -rf "$LOG_DIR"
mkdir -p "$LOG_DIR"
: > "$LOG_DIR/commands.log"
: > "$LOG_DIR/combined.log"

echo "Python: $(python -V)"
echo "PIP_CONSTRAINT=${PIP_CONSTRAINT:-<unset>}"
echo "PROFILE=${PROFILE:-<unset>}"
echo "NUMPY_SPEC=${NUMPY_SPEC:-<unset>}"
echo "GRADIO_SPEC=${GRADIO_SPEC:-<unset>}"
echo "=== ACTIVE CONSTRAINTS FILE ==="
test -s /content/constraints_sonitranslate.txt || { echo "ERROR: /content/constraints_sonitranslate.txt missing"; exit 2; }
sed -n '1,120p' /content/constraints_sonitranslate.txt

# локальные колёса (например, пропатченный omegaconf)
export PIP_FIND_LINKS="/content/_wheels${PIP_FIND_LINKS:+ $PIP_FIND_LINKS}"
export PIP_DISABLE_PIP_VERSION_CHECK=1
export PIP_PROGRESS_BAR=on
export PIP_USE_PEP517=1
export PIP_PREFER_BINARY=1
export PYTHONUNBUFFERED=1

# Режим вывода: compact|full (по умолчанию compact — не душит ноутбук)
export LIVE_MODE=full
export HEARTBEAT_SECS="${HEARTBEAT_SECS:-6}"
export START_LINES="${START_LINES:-50}"

on_fail() {
  echo ""
  echo "===== INSTALL FAILED — LAST 200 LINES OF LOGS ====="
  shopt -s nullglob
  for f in "$LOG_DIR"/*.log; do
    echo "--- $(basename "$f") ---"
    tail -n 200 "$f" || true
  done
  echo "===== DIR TREE ====="
  (set +e; ls -R . | sed 's/^/    /')
}
trap on_fail ERR

# Потоковый запуск: в ноутбук и в общий лог, + отдельный stdout-лог на шаг
run() {
  local cmd="$*"
  local tag="$(echo "$cmd" | sed -E 's/[^A-Za-z0-9_.-]+/_/g' | cut -c1-40)"
  local outlog="$LOG_DIR/step_${tag}.stdout.log"

  echo -e "\n+ $cmd" | tee -a "$LOG_DIR/commands.log"

  # важное: pipefail + возврат кода первой команды (а не tee)
  set -o pipefail
  bash -c "$cmd" 2>&1 | tee -a "$LOG_DIR/combined.log" | tee "$outlog"
  local rc=${PIPESTATUS[0]}
  set +o pipefail

  # небольшая диагностика по свежим логам
  echo "--- tail combined.log ---"; tail -n 25 "$LOG_DIR/combined.log" || true
  echo "--- tail $outlog ---";     tail -n 25 "$outlog" || true
  return $rc
}

# Вспомогалка: короткий error-лог по итогам конкретной операции
extract_errors() {
  local src="$1" dst="$2"
  # ловим варианты вида: "ERROR:" , "[..] ERROR:" , "WARNING:"
  grep -i -E '(^|\[[^]]+\]\s*)(ERROR|WARNING):' "$src" > "$dst" 2>/dev/null || true
}

echo "== ENV =="
echo "SONITRANSLATE: ${SONITRANSLATE_URL} @ ${SONITRANSLATE_REF}"
echo "WHISPERX:      ${WHISPERX_URL} @ ${WHISPERX_REF}"
echo "PYANNOTE:      ${PYANNOTE_URL} @ ${PYANNOTE_REF}"
echo "LOG_DIR:       ${LOG_DIR}"
echo "LIVE_MODE:     ${LIVE_MODE}  (compact shows first ${START_LINES} lines + heartbeat)"
echo "PIP_FIND_LINKS:${PIP_FIND_LINKS:-<unset>}"

# 0) bootstrap
run "python -m pip install -v --upgrade pip wheel uv --log $LOG_DIR/pip_bootstrap.log"
extract_errors "$LOG_DIR/pip_bootstrap.log" "$LOG_DIR/pip_bootstrap_errors.log"
run "python -m pip install -v 'setuptools<81' --log $LOG_DIR/pip_setuptools_compat.log"
extract_errors "$LOG_DIR/pip_setuptools_compat.log" "$LOG_DIR/pip_setuptools_compat_errors.log"

# 1) pre-pin numpy + gradio (без внешних constraints; gradio — без зависимостей)
if [[ -z "${NUMPY_SPEC:-}" || -z "${GRADIO_SPEC:-}" ]]; then
  echo "ERROR: NUMPY_SPEC/GRADIO_SPEC not set (run 07MODE/07AUTO/07CONS)."
  exit 3
fi
run "env -u PIP_CONSTRAINT uv run python -m pip install -v '${NUMPY_SPEC}'  --log $LOG_DIR/pip_prepin_numpy.log"
extract_errors "$LOG_DIR/pip_prepin_numpy.log" "$LOG_DIR/pip_prepin_numpy_errors.log"
run "env -u PIP_CONSTRAINT uv run python -m pip install -v '${GRADIO_SPEC}' --no-deps --log $LOG_DIR/pip_prepin_gradio.log"
extract_errors "$LOG_DIR/pip_prepin_gradio.log" "$LOG_DIR/pip_prepin_gradio_errors.log"

# (опц.) чисто-питоновый docopt — только колесо (чтобы не собирать из sdist)
run "env -u PIP_CONSTRAINT uv run python -m pip install -v --only-binary=:all: 'docopt>=0.6.2' --log $LOG_DIR/pip_docopt.log || true"
extract_errors "$LOG_DIR/pip_docopt.log" "$LOG_DIR/pip_docopt_errors.log"

# 2) CPU/GPU torch-тройка без жёстких версий; задавай в 07DBG при необходимости
TORCH_INDEX_URL="${TORCH_INDEX_URL:-https://download.pytorch.org/whl/cpu}"
TORCH_SPEC="${TORCH_SPEC:-torch torchvision torchaudio}"
run "uv run python -m pip install -v --index-url '${TORCH_INDEX_URL}' ${TORCH_SPEC} --log $LOG_DIR/pip_torch.log"
extract_errors "$LOG_DIR/pip_torch.log" "$LOG_DIR/pip_torch_errors.log"

# 3) чистая копия для установки (branch/tag-safe checkout)
run "rm -rf SoniTranslate_installtest"
run "git clone -q --depth=2 '${SONITRANSLATE_URL}' SoniTranslate_installtest"
cd SoniTranslate_installtest
git fetch -q --depth=2 origin "${SONITRANSLATE_REF}" || true
git checkout -qf "origin/${SONITRANSLATE_REF}" 2>/dev/null || git checkout -qf "refs/tags/${SONITRANSLATE_REF}" 2>/dev/null || echo "[WARN] ref not found; using default HEAD"

# 4) локальные правки ТОЛЬКО в копии requirements
run "sed -i \"s|git+https://github.com/.*/whisperX.git@.*|git+${WHISPERX_URL}@${WHISPERX_REF}|\" requirements_base.txt"
run "sed -i \"s|git+https://github.com/.*/pyannote-audio.git@.*|git+${PYANNOTE_URL}@${PYANNOTE_REF}|\" requirements_base.txt"
run "sed -i -E 's/^torch[^#]*\\+cu[0-9_]+/torch/' requirements_base.txt"
run "sed -i -E 's/(^|[^A-Za-z])TTS==0\\.21\\.1([^A-Za-z]|$)/TTS>=0.22,<0.23/g' requirements*.txt"

# 4b) не даём pip резолвить gradio заново
run "sed -i -E 's/^([[:space:]]*gradio[[:space:]]*==[[:space:]]*[0-9.]+[[:space:]]*)/# (preinstalled) \\1/' requirements_base.txt"
run "sed -i -E 's/^([[:space:]]*gradio[[:space:]]*==[[:space:]]*[0-9.]+[[:space:]]*)/# (preinstalled) \\1/' requirements_extra.txt || true"

# 4c) витрина для дебага
echo "=== REQUIREMENTS SNAPSHOT (after sed) ==="
grep -nE '(^|[[:space:]])(gradio|numpy|websockets|pyannote|docopt)|(^|[[:space:]])(-c|--constraint)[[:space:]]' requirements*.txt || true

# 5) установка зависимостей ПРОЕКТА под constraints
run "env -u PIP_ONLY_BINARY uv run python -m pip install -v -r requirements_base.txt  --constraint /content/constraints_sonitranslate.txt --log $LOG_DIR/pip_req_base.log"
extract_errors "$LOG_DIR/pip_req_base.log" "$LOG_DIR/pip_req_base_errors.log"

# временный даунгрейд pip для fairseq/omegaconf/hydra-core (pip>=24.1 отвергает их метаданные)
run "uv run python -m pip install -v 'pip<24.1' --log $LOG_DIR/pip_downgrade_for_extra.log"
extract_errors "$LOG_DIR/pip_downgrade_for_extra.log" "$LOG_DIR/pip_downgrade_for_extra_errors.log"

run "env -u PIP_ONLY_BINARY uv run python -m pip install -v -r requirements_extra.txt --constraint /content/constraints_sonitranslate.txt --log $LOG_DIR/pip_req_extra.log || true"
extract_errors "$LOG_DIR/pip_req_extra.log" "$LOG_DIR/pip_req_extra_errors.log"

# (опц.) вернуть обратно свежий pip
# run "uv run python -m pip install -v --upgrade pip --log $LOG_DIR/pip_restore_after_extra.log" || true

# 6) sanity-импорты
python - << 'PY'
import sys, os, numpy, torch, gradio
print("python:", sys.version.split()[0])
print("torch:", torch.__version__, "| cuda_available:", torch.cuda.is_available())
print("numpy:", numpy.__version__)
print("gradio:", gradio.__version__)
print("PROFILE:", os.environ.get("PROFILE"))
PY

echo ""
echo "[ok] install test on CPU completed"
echo "Logs saved in: $LOG_DIR"



Python: Python 3.12.11
PIP_CONSTRAINT=<unset>
PROFILE=modern
NUMPY_SPEC=numpy>=2.1,<2.3
GRADIO_SPEC=gradio>=4.30
=== ACTIVE CONSTRAINTS FILE ===
numpy>=2.1,<2.3
scipy>=1.11
opencv-python==4.10.0.84
TTS>=0.22,<0.23
transformers>=4.33,<4.37
sentence-transformers>=2.2,<3.0
tokenizers>=0.13,<0.16
omegaconf==2.0.6
hydra-core==1.0.7
fairseq==0.12.2
gradio>=4.30
== ENV ==
SONITRANSLATE: https://github.com/tekswirl25/SoniTranslate.git @ main
WHISPERX:      https://github.com/tekswirl25/whisperX.git @ cuda_12_x
PYANNOTE:      https://github.com/tekswirl25/pyannote-audio.git @ 3.1.1
LOG_DIR:       /content/_install_logs
LIVE_MODE:     compact  (compact shows first 50 lines + heartbeat)

+ python -m pip install -v --upgrade pip wheel uv --log /content/_install_logs/pip_bootstrap.log
Using pip 25.2 from /usr/local/lib/python3.12/dist-packages/pip (python 3.12)
... working (see /content/_install_logs/combined.log)

+ python -m pip install -v 'setuptools<81' --log /content/_install_logs/pip_setuptoo

CalledProcessError: Command 'b'# [STEP 08/08] DRY INSTALL ON CPU (ISOLATED COPY, VERBOSE LOGS)\n# profile-driven; pins \xd1\x82\xd0\xbe\xd0\xbb\xd1\x8c\xd0\xba\xd0\xbe \xd0\xb8\xd0\xb7 07MODE/07AUTO/07CONS; \xd0\xb1\xd0\xb5\xd0\xb7 \xd1\x85\xd0\xb0\xd1\x80\xd0\xb4\xd0\xba\xd0\xbe\xd0\xb4\xd0\xbe\xd0\xb2 \xd0\xb7\xd0\xb4\xd0\xb5\xd1\x81\xd1\x8c\nset -euo pipefail\n\nLOG_DIR="/content/_install_logs"\n\n# --- \xd0\x9b\xd0\x9e\xd0\x93\xd0\x98: \xd1\x80\xd0\xbe\xd1\x82\xd0\xb0\xd1\x86\xd0\xb8\xd1\x8f (\xd0\xbf\xd0\xbe \xd0\xb6\xd0\xb5\xd0\xbb\xd0\xb0\xd0\xbd\xd0\xb8\xd1\x8e) \xd0\xb8 \xd0\xa7\xd0\x98\xd0\xa1\xd0\xa2\xd0\xab\xd0\x99 \xd0\xa1\xd0\xa2\xd0\x90\xd0\xa0\xd0\xa2 ---\n# ROTATE_LOGS=1 \xd1\x81\xd0\xbe\xd1\x85\xd1\x80\xd0\xb0\xd0\xbd\xd0\xb8\xd1\x82 \xd0\xbf\xd1\x80\xd0\xb5\xd0\xb4\xd1\x8b\xd0\xb4\xd1\x83\xd1\x89\xd1\x83\xd1\x8e \xd0\xbf\xd0\xb0\xd0\xbf\xd0\xba\xd1\x83 \xd0\xbb\xd0\xbe\xd0\xb3\xd0\xbe\xd0\xb2 \xd1\x81\xd0\xbe \xd1\x88\xd1\x82\xd0\xb0\xd0\xbc\xd0\xbf\xd0\xbe\xd0\xbc \